In [10]:
import numpy as np
import torch
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from scripts.data_auto import DataAuto

In [11]:
# make df with column names year, month, decimal date, SSN, M1, M2, M3, M4, M5 from file 5c_obs_sim_v1.dat
df = pd.read_csv('5c_obs_sim_v1.dat', sep='\s+', header=None, names=['year', 'month', 'decimal_date', 'SSN', 'M1', 'M2', 'M3', 'M4', 'M5'])

# normalize data to be between 0 and 1 using max value from SSN
M = df['SSN'].max()
df['SSN'] = df['SSN'] / M
for i in range(1, 6):
    df[f'M{i}'] = df[f'M{i}'] / M

df.tail()

,year,month,decimal_date,SSN,M1,M2,M3,M4,M5
643,2018,8,2018.623,0.028768,0.028838,0.007422,0.071437,0.138527,0.159606
644,2018,9,2018.705,0.027909,0.029266,0.007966,0.073743,0.135698,0.167333
645,2018,10,2018.790,0.029197,0.029564,0.008762,0.075843,0.132842,0.176335
646,2018,11,2018.873,0.028768,0.030029,0.009605,0.078106,0.129633,0.184630
647,2018,12,2018.958,0.025762,0.030656,0.010354,0.080041,0.126078,0.192651


In [12]:
model_series = df['M1'].values
observed_series = df['SSN'].values

# make dates into datetime objects from year and month in format yyyy-mm
dates = pd.to_datetime(df[['year', 'month']].assign(day=1))
dates

0     1965-01-01
1     1965-02-01
2     1965-03-01
3     1965-04-01
4     1965-05-01
         ...    
643   2018-08-01
644   2018-09-01
645   2018-10-01
646   2018-11-01
647   2018-12-01
Length: 648, dtype: datetime64[ns]

# AutoLin

In [4]:

# Генерация данных
data = DataAuto(model_series=model_series, observed_series=observed_series, previous_values=4)

train_data_X = data.X[:360].numpy()
train_data_y = data.y[:360].numpy()

test_data_X = data.X[360:].numpy()
test_data_y = data.y[360:].numpy()

# Обучение линейной регрессии
linear_model = LinearRegression()
linear_model.fit(train_data_X, train_data_y)


LinearRegression()

In [5]:
linear_model.coef_

array([-0.46322024,  0.871093  , -0.15623218, -0.5515664 ,  0.30891836,
       -0.45397672,  1.275235  , -2.05173   ,  2.2170548 ], dtype=float32)

In [6]:

# Функция для авторегрессионного предсказания
def autoregressive_prediction(model, data_X, initial_index, steps):
    preds = []
    current_X = data_X[initial_index].copy()
    
    for step in range(steps):
        # Предсказание на основе текущих данных
        pred = model.predict(current_X.reshape(1, -1))[0]
        preds.append(pred)
        
        # Обновление входных данных
        model_s = data_X[initial_index + step + 1, :5] if initial_index + step + 1 < len(data_X) else np.zeros(5)
        obs_s = current_X[-3:]
        obs_s = np.append(obs_s, pred)
        current_X = np.concatenate((model_s, obs_s[-4:]))
    
    return preds

# Вычисление RMSE для прогноза на каждый месяц
def compute_rmse_autoregressive(model, test_X, test_y, steps):
    rmses = []
    for step in range(steps):  # Включаем прогноз на 1 месяц вперед
        step_preds = []
        step_targets = []
        for i in range(len(test_X) - step):
            preds = autoregressive_prediction(model, test_X, i, step + 1)
            step_preds.append(preds[-1])  # Берем только последний месяц
            step_targets.append(test_y[i + step])
        
        # Вычисление RMSE для данного шага
        step_rmse = np.sqrt(mean_squared_error(step_targets, step_preds))
        rmses.append(step_rmse)
    return rmses


In [7]:

# Получение RMSE для прогноза на 18 месяцев
steps = 18
rmses = compute_rmse_autoregressive(linear_model, test_data_X, test_data_y, steps)

# Вывод результатов
for month, rmse in enumerate(rmses, start=1):
    print(f"Month {month}: RMSE = {rmse * M:.4f}")



Month 1: RMSE = 1.3107
Month 2: RMSE = 3.1802
Month 3: RMSE = 4.7544
Month 4: RMSE = 6.2001
Month 5: RMSE = 7.6669
Month 6: RMSE = 9.1947
Month 7: RMSE = 10.7422
Month 8: RMSE = 12.2051
Month 9: RMSE = 13.7142
Month 10: RMSE = 15.2821
Month 11: RMSE = 16.8344
Month 12: RMSE = 18.3905
Month 13: RMSE = 19.7611
Month 14: RMSE = 20.9241
Month 15: RMSE = 21.9983
Month 16: RMSE = 22.9796
Month 17: RMSE = 23.8572
Month 18: RMSE = 24.6390


In [9]:
rmses = np.array(rmses)
# save
np.save('rmses_linreg.npy', rmses)

# 18-forwrad linreg

In [13]:
from scripts import Data18Forward

In [14]:
# Генерация данных
data = Data18Forward(model_series=model_series, observed_series=observed_series, previous_values=4)

train_data_X = data.X[:360].numpy()
train_data_y = data.y[:360].numpy()

test_data_X = data.X[360:].numpy()
test_data_y = data.y[360:].numpy()

# Обучение линейной регрессии
linear_model = LinearRegression()
linear_model.fit(train_data_X, train_data_y)


LinearRegression()

In [15]:
def compute_rmse_18forward(model, test_X, test_y):
    preds = model.predict(test_X)
    rmse = np.sqrt(mean_squared_error(test_y, preds))
    return rmse

# Вычисление RMSE для прогноза на 18 месяцев
rmse = compute_rmse_18forward(linear_model, test_data_X, test_data_y)
rmse

0.065806426

In [24]:
preds = linear_model.predict(test_data_X)
rmses = np.sqrt(mean_squared_error(test_data_y, preds, multioutput='raw_values'))
rmses

array([0.0057445 , 0.01384314, 0.020789  , 0.02710046, 0.03334456,
       0.03978192, 0.04632557, 0.05246791, 0.05850363, 0.06460066,
       0.07090466, 0.07743138, 0.0829308 , 0.08706541, 0.09048857,
       0.09346128, 0.09612259, 0.09861259], dtype=float32)

In [27]:
# save rmses
np.save('rmses_18forward.npy', rmses)

# x18 linreg

In [26]:
from scripts import Datax18

In [31]:
rmses = []
for horizon in range(1, 19):
    data = Datax18(model_series=model_series, observed_series=observed_series, previous_values=4, horizon=horizon)
    
    train_data_X = data.X[:360].numpy()
    train_data_y = data.y[:360].numpy()

    test_data_X = data.X[360:].numpy()
    test_data_y = data.y[360:].numpy()
    
    model = LinearRegression()
    model.fit(train_data_X, train_data_y)
    
    preds = model.predict(test_data_X)
    rmse = np.sqrt(mean_squared_error(test_data_y, preds))
    rmses.append(rmse)
    
rmses = np.array(rmses)
    

In [36]:
train_data_X.shape

(360, 26)

In [34]:
# save rmses
np.save('rmses_x18.npy', rmses)

# NN

In [4]:
from scripts import DataAuto, NARXAuto

In [5]:
data = DataAuto(model_series=model_series, observed_series=observed_series, previous_values=4)

train_data_X = data.X[:360]
train_data_y = data.y[:360]

test_data_X = data.X[360:]
test_data_y = data.y[360:]

In [7]:
model = NARXAuto(input_size=9, hidden_sizes=[20, 13, 4], output_size=1, M=M)

In [8]:
trainloader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(train_data_X, train_data_y), batch_size=64, shuffle=True)

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

NARXAuto(
  (net): Sequential(
    (0): Linear(in_features=9, out_features=20, bias=True)
    (1): Linear(in_features=20, out_features=13, bias=True)
    (2): Linear(in_features=13, out_features=4, bias=True)
    (3): Linear(in_features=4, out_features=1, bias=True)
  )
)

In [19]:
train_losses, rmses = model.train_model(trainloader, test_data_X, test_data_y, epochs=10000, lr=0.001)

Training: 100%|██████████| 10000/10000 [02:31<00:00, 65.95epoch/s]


In [20]:
for i, r in enumerate(rmses[-1]):
    print(f"Month {i + 1}: RMSE = {r:.4f}")

Month 1: RMSE = 1.8025
Month 2: RMSE = 4.2380
Month 3: RMSE = 6.7274
Month 4: RMSE = 9.2798
Month 5: RMSE = 11.8832
Month 6: RMSE = 14.4318
Month 7: RMSE = 16.8286
Month 8: RMSE = 19.0602
Month 9: RMSE = 21.1682
Month 10: RMSE = 23.1756
Month 11: RMSE = 25.1023
Month 12: RMSE = 26.9403
Month 13: RMSE = 28.6573
Month 14: RMSE = 30.2881
Month 15: RMSE = 31.8928
Month 16: RMSE = 33.4828
Month 17: RMSE = 35.0519
Month 18: RMSE = 36.6019
